# Introduction to image classification using camera trap images
***

## 1. Set up environment
***

Next, let's import some necessary libraries of the usual suspects:

In [14]:
# Data Science libraries
import pandas as pd # data processing, CSV file I/O
import numpy as np # linear algebra

# Visualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

# Tensorflow and Keras libraries
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy

# Metrics
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# System libraries
import os
import platform
import shutil
import datetime

# # CLI and Python library for interacting with the Weights and Biases API
# import wandb
# from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

In [2]:
os.chdir(r'C:\GitHub\CameraTrap-Animal-Classification')

In [3]:
# Reproducability
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

In [ ]:
!nvcc --version
# !nvidia-smi

In [ ]:
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow was built with CUDA (GPU) support:", tf.test.is_built_with_cuda())
print("TensorFlow version:", tf.__version__)
print("Python version:", platform.python_version())

In [ ]:
# dataset_path = r'C:\GitHub\CameraTrap-Animal-Classification\data\raw'
dataset_path = 'data/raw'
os.makedirs(dataset_path, exist_ok=True)

train_features = pd.read_csv(os.path.join(dataset_path, 'train_features.csv'), index_col="id")
test_features = pd.read_csv(os.path.join(dataset_path, 'test_features.csv'), index_col="id")
train_labels = pd.read_csv(os.path.join(dataset_path, 'train_labels.csv'), index_col="id")

## 2. Build the model
***

### Define parameters

In [6]:
IMG_HEIGHT = 360 # 224
IMG_WIDTH = 640 # 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
CHANNELS = 3
BATCH_SIZE = 16
EPOCHS = 30
BASE_LEARNING_RATE = 1e-3
NUM_CLASSES = 8

### Log a run to start tracking system metrics and console logs in Weights&Biases.

In [7]:
# wandb.login(key='')

In [8]:
# run = wandb.init(
#     project="Conservision_Practice_Area_Image_Classification"
# )

In [13]:
# wandb.config.update({
#     "IMG_HEIGHT": IMG_HEIGHT,
#     "IMG_WIDTH": IMG_WIDTH,
#     "IMG_SIZE": IMG_SIZE,
#     "CHANNELS": CHANNELS,
#     "BATCH_SIZE": BATCH_SIZE,
#     "EPOCHS": EPOCHS,
#     "BASE_LEARNING_RATE": BASE_LEARNING_RATE,
#     "NUM_CLASSES": NUM_CLASSES
# })

### Data pre-processing

In [ ]:
from keras.applications.resnet import preprocess_input
# from keras.applications.efficientnet import preprocess_input
# from keras.applications.convnext import preprocess_input

dataset_path = r'data/raw'
os.makedirs(dataset_path, exist_ok=True)
species_labels = sorted(train_labels.columns.unique())
train_dir = os.path.join(dataset_path, 'train')
valid_dir = os.path.join(dataset_path, 'validation')

func_preprocess_input = preprocess_input
train_datagen = ImageDataGenerator(
    preprocessing_function=func_preprocess_input,
    # horizontal_flip=True,
    # rotation_range=5,
    # shear_range=0.1,
    # zoom_range=[0.9, 1.0],
    # brightness_range=[0.9, 1.1],
    # fill_mode='nearest'
)

train_ds = train_datagen.flow_from_directory(
    directory=train_dir,
    color_mode='rgb',
    class_mode='categorical',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

valid_ds = validation_datagen.flow_from_directory(
    directory=valid_dir,
    color_mode='rgb',
    class_mode='categorical',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42
)

In [ ]:
# Inspect a batch from train_ds
images, labels = next(iter(train_ds))
print("Train images shape and type:", images.dtype, images.shape)
print("Train labels shape and type:", labels.dtype, labels.shape)

# Inspect a batch from val_dataset
images, labels = next(iter(valid_ds))
print("Validation images shape and type:", images.dtype, images.shape)
print("Validation labels shape and type:", labels.dtype, labels.shape)

### Visualization augmentation data

In [ ]:
image_id = 'ZJ004793' # 'ZJ000048'

train_features = pd.read_csv(os.path.join(dataset_path, 'train_features.csv'), index_col="id")

image_filepath = train_features.loc[image_id, 'filepath']
image_fullpath = os.path.join(dataset_path, image_filepath)

original_image = Image.open(image_fullpath)
original_image_np = np.array(original_image)

augmenters = {
    "Horizontal Flip": ImageDataGenerator(horizontal_flip=True),
    "Rotation": ImageDataGenerator(rotation_range=5),
    "Shear": ImageDataGenerator(shear_range=0.1),
    "Zoom": ImageDataGenerator(zoom_range=[0.9, 1.0]),
    "Brightness": ImageDataGenerator(brightness_range=[0.9, 1.1]),
    # "Fill Mode": ImageDataGenerator(fill_mode='nearest')
}

augmenters_names = {
    "Horizontal Flip": "Example 2: horizontal_flip",
    "Rotation": "Example 3: rotation_range",
    "Shear": "Example 4: shear_range",
    "Zoom": "Example 5: zoom_range",
    "Brightness": "Example 6: brightness_range",
    # "Fill Mode": "Example 7: fill_mode"
}

# Function to augment image with optional seed for reproducibility
def augments_image(image, datagen, seed=None):
    image = np.expand_dims(image, axis=0)
    iterator = datagen.flow(image, batch_size=1, seed=seed)  # Use seed if provided
    augmented_image = iterator.next()[0].astype('uint8')
    return augmented_image

# Number of augmentations
num_augmentations = len(augmenters)
num_columns = 2
num_rows = (num_augmentations + 1 + num_columns - 1) // num_columns

fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, num_rows * 5))
axes = axes.flatten()

# Display original image
axes[0].imshow(original_image_np)
axes[0].set_title("Example 1: original")
axes[0].axis('on')

# Decide if you want reproducibility
use_seed = True  # Set to False if you don't want consistent results
seed_value = 34  # Seed value to use if reproducibility is desired

# Display augmented images
for ax, (aug_name, datagen) in zip(axes[1:], augmenters.items()):  # Start from the second subplot
    augmented_image_np = augments_image(original_image_np, datagen, seed_value if use_seed else None)
    ax.imshow(augmented_image_np)
    ax.set_title(augmenters_names[aug_name])
    ax.axis('on')

# Hide unused axes
for ax in axes[num_augmentations + 1:]:
    ax.axis('on')

plt.tight_layout()
plt.show()

### Define pretrained model

In [ ]:
from keras.applications import ResNet101, EfficientNetB0, EfficientNetB6, ConvNeXtSmall, ConvNeXtBase

# choose base network
network = 'ResNet101'

shape = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)

if network == 'ResNet101':
    pretrained_model = ResNet101(include_top=False, weights='imagenet', input_shape=shape)
elif network == 'EfficientNetB0':
    pretrained_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=shape)
elif network == 'EfficientNetB6':
    pretrained_model = EfficientNetB6(include_top=False, weights='imagenet', input_shape=shape)
elif network == 'ConvNeXtSmall':
    pretrained_model = ConvNeXtSmall(include_top=False, weights='imagenet', input_shape=shape)
elif network == 'ConvNeXtBase':
    pretrained_model = ConvNeXtBase(include_top=False, weights='imagenet', input_shape=shape)
else:
    print('Network name does not exist')

pretrained_model.trainable = False # default is True
pretrained_model.summary()

In [ ]:
for i, layer in enumerate(pretrained_model.layers):
    print(i, layer.name, layer.trainable)

# for layer in pretrained_model.layers[331:]:
#     layer.trainable = True

In [ ]:
model = Sequential([

    pretrained_model,

    # Flatten the output of the pretrained model
    GlobalAveragePooling2D(),

    # Add custom layers on top of the pretrained model
    Dense(units=128, activation='relu'),
    Dropout(rate=0.1),
    
    Dense(units=64, activation='relu'),
    Dropout(rate=0.1),
    
    Dense(units=32, activation='relu'),
    Dropout(rate=0.1),
    
    Dense(units=16, activation='relu'),
    Dropout(rate=0.1),
    
    Dense(units=NUM_CLASSES, activation='softmax')
])

model.summary()

In [ ]:
# pretrained_model.trainable = False
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

## 3. Training

In [19]:
model_name = pretrained_model.name
timestamp = datetime.datetime.now().strftime("%H%M-%d%m%Y")

checkpoint_loss = ModelCheckpoint(filepath=(f'model_best_loss_{model_name}_{timestamp}.keras'),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False)

checkpoint_acc = ModelCheckpoint(filepath=(f'model_best_acc_{model_name}_{timestamp}.keras'),
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False)

# lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
#                               factor=0.1,
#                               patience=3,
#                               verbose=1,
#                               min_lr=1e-6)

# early_stopping = EarlyStopping(
#     min_delta=0.001, # minimium amount of change to count as an improvement
#     patience=5, # how many epochs to wait before stopping
#     restore_best_weights=True,
# )

# wandb_logger = WandbMetricsLogger(log_freq=1)

# callbacks = [checkpoint_loss, checkpoint_acc, wandb_logger]
callbacks = [checkpoint_loss, checkpoint_acc]

In [20]:
model.compile(optimizer=Adam(learning_rate=BASE_LEARNING_RATE),
              loss=CategoricalCrossentropy(name="categorical_crossentropy"),
              metrics=CategoricalAccuracy(name="accuracy"))

In [ ]:
history = model.fit(train_ds,
                    epochs=EPOCHS,
                    callbacks=callbacks,
                    validation_data=valid_ds,
                    max_queue_size=12,
                    workers=4)

In [ ]:
# # Close the W&B run
# run.finish()

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
history_df = pd.DataFrame(history_dict)
new_index = []
# Generate the new index labels using a for loop
for i in range(1, len(history_df) + 1):
    new_index.append('Epoch ' + str(i))
history_df.index = new_index
history_df

In [ ]:
from matplotlib.ticker import MultipleLocator

# Extract values from the training history
history_dict = history.history
training_loss = history_dict['loss']
validation_loss = history_dict['val_loss']
training_accuracy = history_dict['accuracy']
validation_accuracy = history_dict['val_accuracy']

def training_plot(metrics, history):
    num_epochs = len(history.history[metrics[0]])  # Number of epochs
    epochs = range(1, num_epochs + 1)  # Create a range object for the x-axis values

    f, ax = plt.subplots(2, len(metrics)//2, figsize=(5 * len(metrics), 12))

    for idx, metric in enumerate(metrics):
        training_metric = history.history[metric]
        validation_metric = history.history['val_' + metric]

        ax[idx].plot(epochs, training_metric, ls='-', marker='o', color='red', label='train_' + metric)
        ax[idx].plot(epochs, validation_metric, ls='--', marker='o', color='blue', label='val_' + metric)

        ax[idx].set_xlabel("epoka")
        ax[idx].set_ylabel(metric)
        ax[idx].legend()
        ax[idx].grid()
        ax[idx].set_title(f'{metric.capitalize()} - trening i walidacja')
        ax[idx].set_xticks(epochs)  # Set the x-ticks to match the epochs
        ax[idx].set_xlim([0.6, num_epochs + 0.4])  # Adding a margin to the left and right of the plot
        # ax[idx].yaxis.set_major_locator(MultipleLocator(0.05)) # Set y-axis ticks to have intervals of 0.1

    plt.tight_layout()
    plt.show()

training_plot(['loss', 'accuracy'], history)

### Save model and history

In [ ]:
def generate_model_name(base_name, model_name, epoch=None, val_loss=None, val_accuracy=None):
    name_parts = [base_name,
                  model_name,
                  f"epoch_{epoch}" if epoch is not None else "",
                  f"val_loss_{val_loss:.4f}" if val_loss is not None else "",
                  f"val_acc_{val_accuracy:.4f}" if val_accuracy is not None else ""]

    return "_".join([part for part in name_parts if part])

In [ ]:
best_val_loss_idx = history.history['val_loss'].index(min(history.history['val_loss']))
# best_val_acc_idx = history.history['val_acc'].index(min(history.history['val_acc']))
best_val_loss = history.history['val_loss'][best_val_loss_idx]
best_val_acc = history.history['val_accuracy'][best_val_loss_idx]

# Save training history
history_model_df = pd.DataFrame(history_dict)
history_file_name = generate_model_name(base_name='history_best_loss',
                                        model_name=model_name,
                                        epoch=best_val_loss_idx + 1,
                                        val_loss=best_val_loss,
                                        val_accuracy=best_val_acc) + '.csv'

history_model_df.to_csv(history_file_name, index=False)

final_model_name = generate_model_name(
    base_name='model_saved',
    model_name=model_name,
    epoch=None,
    val_loss=None,
    val_accuracy=None)

model.save(f"{final_model_name}.keras")

### Load history and model (plot)

In [ ]:
# Path to the CSV file
history_path = os.path.join("modele_wyniki\ResNet-101\Model2(StratifiedGroupKFold)\history_best_loss_resnet101_epoch_1_val_loss_1.6934_val_acc_0.3827.csv")
history_load = pd.read_csv(history_path)

# Adding a column 'epoch' with values 'Epoch 1', 'Epoch 2', etc.
history_load.insert(0, 'epoch', ['Epoch ' + str(i) for i in range(1, len(history_load) + 1)])

# Changing the indexing to start from 1
history_load.index = range(1, len(history_load) + 1)
history_load

In [ ]:
from custom_layers_ConvNeXt import LayerScale, StochasticDepth

model_path = os.path.join("modele_wyniki\ResNet-101\Model2(StratifiedGroupKFold)\model_best_loss_resnet101_1802-23082024.keras")

load_model = tf.keras.models.load_model(model_path)
# load_model = tf.keras.models.load_model(model_path, custom_objects={'LayerScale': LayerScale, 'StochasticDepth': StochasticDepth})

In [ ]:
from matplotlib.ticker import MultipleLocator

def training_plot(metrics, history_df, fontsize=18):
    num_epochs = len(history_df)  # Number of epochs
    epochs = range(1, num_epochs + 1)  # Create a range object for the x-axis values, starting from 1
    tick_intervals = range(0, num_epochs + 1, 2)  # Set tick intervals at 0, 2, 4, etc.

    f, ax = plt.subplots(1, len(metrics), figsize=(8 * len(metrics), 7))

    for idx, metric in enumerate(metrics):
        training_metric = history_df[metric]
        validation_metric = history_df['val_' + metric]

        # Use markers for training and validation metrics
        ax[idx].plot(epochs, training_metric, ls='-', marker='o', color='red', label='train_' + metric) # red
        ax[idx].plot(epochs, validation_metric, ls='--', marker='o', color='blue', label='validation_' + metric) # blue

        # Set labels and title with specified font size
        ax[idx].set_xlabel("epoka", fontsize=fontsize)
        ax[idx].set_ylabel('wartość ' + metric, fontsize=fontsize)
        ax[idx].legend(fontsize=fontsize)
        ax[idx].grid()
        ax[idx].set_title(f'{metric.capitalize()} - trening i walidacja', fontsize=fontsize+2)
        ax[idx].set_xticks(tick_intervals)  # Set the x-ticks to every 2 epochs starting from 0
        ax[idx].set_xlim([0, num_epochs + 0.4])  # Set x-axis limits to include 0

        # ax[idx].yaxis.set_major_locator(MultipleLocator(0.05))
        # Adjust the tick label size
        ax[idx].tick_params(axis='both', which='major', labelsize=fontsize)

    plt.tight_layout()
    plt.show()

training_plot(['loss', 'accuracy'], history_load, fontsize=16)

## 4. Evaluation on validation set
***

### Make predictions labels distribution

In [ ]:
validation_loss, validation_accuracy = load_model.evaluate(valid_ds, max_queue_size=14, workers=4)
print(f"Validation Loss: {validation_loss:.4f}")
print(f"Validation Accuracy: {validation_accuracy:.4f}")

In [ ]:
# predictions = load_model.predict(valid_ds, max_queue_size=14, workers=4)

# val_preds_df = pd.DataFrame(predictions, columns=species_labels)
# # Extract filenames without path and extension
# image_ids = [os.path.splitext(os.path.basename(file_path))[0] for file_path in valid_ds.filenames]
# val_preds_df.index = image_ids

# val_preds_df = val_preds_df.round(6)
# val_preds_df

In [ ]:
# Generate predictions using the validation generator
predictions = load_model.predict(valid_ds, max_queue_size=14, workers=4)
# Create a DataFrame with the predictions
val_preds_df = pd.DataFrame(predictions, columns=species_labels)
# Extract the species labels (class names) from the generator
species_labels = list(valid_ds.class_indices.keys())
# Extract image IDs from the generator's filenames
image_ids = [os.path.splitext(os.path.basename(file_path))[0] for file_path in valid_ds.filenames]
val_preds_df.index = image_ids
val_preds_df.index.name = None
# Determine the predicted classes and their labels
val_preds_df['predicted_label'] = val_preds_df.idxmax(axis=1)
# Map the true class labels using the filenames directly
val_preds_df['true_label'] = [os.path.basename(os.path.dirname(file_path)) for file_path in valid_ds.filenames]

val_preds_df = val_preds_df.round(6)
val_preds_df

### Classification report

In [ ]:
true_classes = valid_ds.classes
predicted_classes = np.argmax(predictions, axis=1)
class_labels = list(valid_ds.class_indices.keys())

report = classification_report(y_true=true_classes,
                               y_pred=predicted_classes,
                               target_names=class_labels,
                               output_dict=True,
                               zero_division=0)

report_df = pd.DataFrame(report).transpose()
report_df

### Confusion matrix

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

cm_display = ConfusionMatrixDisplay.from_predictions(
    y_true=true_classes,
    y_pred=predicted_classes,
    display_labels=species_labels,
    normalize=None,
    ax=ax,
    xticks_rotation=90,
    colorbar=True,
    cmap='viridis'
)

ax.set_ylabel('Prawdziwe etykiety', fontsize=7, fontweight='bold')
ax.set_xlabel('Prognozowane etykiety', fontsize=7, fontweight='bold')
plt.title('Macierz pomyłek', fontsize=9, fontweight='bold')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))

cm_display = ConfusionMatrixDisplay.from_predictions(
    y_true=true_classes,
    y_pred=predicted_classes,
    display_labels=species_labels,
    normalize='true',
    ax=ax,
    xticks_rotation=90,
    colorbar=True,
    cmap='viridis'
)

ax.set_ylabel('Prawdziwe etykiety', fontsize=7, fontweight='bold')
ax.set_xlabel('Prognozowane etykiety', fontsize=7, fontweight='bold')
plt.title('Macierz pomyłek', fontsize=9, fontweight='bold')

plt.show()

### Predicted labels distribution

In [ ]:
def load_data(dataset_path, subdir, features_df):
    """
    Function loads data from a directory and creates DataFrames for features (x_data) and labels (y_data).

    Parameters:
    - dataset_path: base path to the dataset directory
    - subdir: subdirectory from which data should be loaded ('train' or 'validation')
    - features_df: DataFrame containing additional features (e.g., 'site')

    Returns:
    - x_data: DataFrame containing file paths and features (site)
    - y_data: DataFrame with one-hot encoded labels
    """
    dir_path = os.path.join(dataset_path, subdir)

    # Creating a list of dictionaries, where each dictionary contains file, site, and class information
    data = [
        {
            'id': os.path.splitext(filename)[0],  # Extracting file ID without the extension
            'filepath': os.path.join(subdir, filename),  # File path in the subdirectory
            # Retrieving the 'site' value from the DataFrame based on the ID
            'site': features_df.loc[os.path.splitext(filename)[0], 'site'] if os.path.splitext(filename)[0] in features_df.index else None,
            'label': class_label  # Class name (e.g., 'antelope_duiker', 'bird')
        }
        for class_label in os.listdir(dir_path)  # Iterating through each class in the subdirectory
        if os.path.isdir(os.path.join(dir_path, class_label))  # Checking if it is a directory (class)
        for filename in os.listdir(os.path.join(dir_path, class_label))  # Iterating through each file in the given class
    ]

    # Creating a DataFrame from the list of dictionaries and setting 'id' as the index
    data_df = pd.DataFrame(data).set_index('id')
    # Separating 'filepath' and 'site' columns as x_data
    x_data = data_df[['filepath', 'site']]
    # Encoding labels with one-hot encoding (e.g., columns for each class) and converting to integers
    y_data = pd.get_dummies(data_df['label']).astype(int)

    return x_data, y_data

x_train, y_train = load_data(dataset_path, 'train', train_features)
x_val, y_val = load_data(dataset_path, 'validation', train_features)

In [ ]:
# True labels distribution in the training set
print("True labels (train):")
print(y_train.idxmax(axis=1).value_counts())

In [ ]:
# Print the predicted labels distribution using idxmax on a copy with only numeric columns
preds_only_df = val_preds_df[species_labels].copy()
print("Predicted labels (validation):")
print(preds_only_df.idxmax(axis=1).value_counts())

In [ ]:
# Print the true labels distribution in the validation set
print("True labels (validation):")
print(y_val.idxmax(axis=1).value_counts())

## 5. Submission test_features
***

In [ ]:
test_features_dir = os.path.join(dataset_path, 'test_features')

temp_test_dir = os.path.join(test_features_dir, 'temp_test_dir')
os.makedirs(temp_test_dir, exist_ok=True)
test_images_dir = os.path.join(temp_test_dir, 'test_images')
os.makedirs(test_images_dir, exist_ok=True)

for file_name in os.listdir(test_features_dir):
    file_path = os.path.join(test_features_dir, file_name)
    if os.path.isfile(file_path):
        shutil.copy(file_path, os.path.join(test_images_dir, file_name))

func_preprocess_input = preprocess_input

test_datagen = ImageDataGenerator(preprocessing_function=func_preprocess_input)

test_ds = test_datagen.flow_from_directory(
    directory=temp_test_dir,
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=42
)

In [ ]:
file_paths_no_labels = [os.path.join(test_features_dir, f) for f in os.listdir(test_features_dir) if f.lower().endswith('.jpg')]
file_paths_no_labels.sort()

df_test = pd.DataFrame({'filename': file_paths_no_labels})

test_predictions = load_model.predict(test_ds, max_queue_size=14, workers=4)
species_labels = sorted(train_labels.columns.unique())
test_preds_df = pd.DataFrame(test_predictions, columns=species_labels)
test_preds_df.index = [os.path.splitext(os.path.basename(filename))[0] for filename in test_ds.filenames]
test_preds_df = test_preds_df.round(6)
test_preds_df

In [ ]:
# shutil.rmtree(temp_test_dir)
submission_format = pd.read_csv(os.path.join(dataset_path, 'submission_format.csv'), index_col="id")

assert all(test_preds_df.index == submission_format.index)
assert all(test_preds_df.columns == submission_format.columns)

test_preds_df.to_csv("submission_df.csv")

In [ ]:
history_dict = history.history
history_dict.keys()